In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

C:\Users\aajna\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\aajna\Anaconda3\envs\tensorflow\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an 

In [2]:
# fix random seed for reproducibility
np.random.seed(7)

In [3]:
# load the dataset
dataframe = pd.read_csv('Google_Stock_Price_Train.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')

In [4]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [5]:
# split into train and test sets
train_size = int(len(dataset) * 1)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

1255 0


In [6]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [7]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
print(trainX)

[[0.08581364]
 [0.0970124 ]
 [0.09433365]
 ...
 [0.96294373]
 [0.96123224]
 [0.95475847]]


In [8]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

In [9]:
# create and fit the LSTM network
look_back = 1
model = Sequential()
model.add(GRU(100, batch_input_shape=(1,1, look_back),stateful = True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
 - 3044s - loss: 0.0070
Epoch 2/10
 - 2709s - loss: 4.0164e-04
Epoch 3/10
 - 2702s - loss: 4.4727e-04
Epoch 4/10
 - 2708s - loss: 4.7123e-04
Epoch 5/10
 - 2701s - loss: 4.0493e-04
Epoch 6/10
 - 2697s - loss: 4.2129e-04
Epoch 7/10
 - 2717s - loss: 4.1539e-04
Epoch 8/10
 - 2707s - loss: 4.3991e-04
Epoch 9/10
 - 2711s - loss: 3.7762e-04
Epoch 10/10
 - 2718s - loss: 4.0296e-04


In [10]:
print('Model predict')
#pred = model.predict(np.reshape([0.98,0.88], (1,1,2)))
pred = model.predict(np.reshape(0.98, (1,1,1)))
print(pred)
pred = model.predict(np.reshape(0.88, (1,1,1)))
print(pred)
model.reset_states()
pred = model.predict(np.reshape(0.98, (1,1,1)))
print(pred)
pred = model.predict(np.reshape(0.88, (1,1,1)))
print(pred)

Model predict
[[0.95817435]]
[[0.8596926]]
[[0.96858305]]
[[0.83239174]]


In [11]:
print(scaler.inverse_transform(pred))

[[726.58057]]


In [12]:
for layer in model.layers:
    if 'GRU' in str(layer):
        w = layer.get_weights()
        #print (np.array(w[0])).shape
        #print(w)
        #if 'activation' in str(layer):
        #print layer.get_config()
        #print K.get_value(layer.bias_i)

        
for layer in model.layers:
    if 'Dense' in str(layer):
        w = layer.get_weights()
        print(w[0])
        print(w[1])
        W = w[0]
        B = w[1]
        print(W)
        print(B)
        with open('W.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in w:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        #print (np.array(w[0])).shape
        #print layer.get_config()
print(W[0],W[1])
print(B[0])

[[-1.25736475e-01]
 [ 1.55252561e-01]
 [ 1.33633450e-01]
 [ 1.91064633e-03]
 [ 1.12503618e-02]
 [ 6.17386308e-03]
 [ 1.85944401e-02]
 [ 1.70966282e-01]
 [-4.45786081e-02]
 [ 9.67784226e-02]
 [-2.73136259e-03]
 [ 4.07473650e-03]
 [ 6.26210272e-02]
 [ 1.16005191e-03]
 [ 1.71554065e-03]
 [-1.11743875e-01]
 [ 1.19454034e-01]
 [ 1.74502693e-02]
 [-1.20566346e-01]
 [-9.50467214e-02]
 [ 2.48098254e-01]
 [-1.52729765e-01]
 [ 1.76605865e-01]
 [ 1.86098386e-02]
 [ 4.01315599e-04]
 [-8.39074403e-02]
 [-1.98375344e-01]
 [ 7.77108669e-02]
 [ 1.78209320e-01]
 [-1.60121307e-01]
 [ 8.20115581e-03]
 [-9.05977115e-02]
 [ 1.01256728e-01]
 [ 3.85918841e-03]
 [-2.04835013e-01]
 [-4.69374354e-05]
 [-1.12192541e-01]
 [-2.39957892e-03]
 [ 1.88730133e-03]
 [-1.26312271e-01]
 [-7.36376597e-03]
 [ 1.24601446e-01]
 [-4.25172225e-02]
 [ 5.00662066e-02]
 [-6.38865978e-02]
 [-3.02538257e-02]
 [ 1.54384375e-01]
 [-1.60541520e-01]
 [-2.69369394e-01]
 [-1.57068332e-03]
 [ 1.96783662e-01]
 [-2.42017150e-01]
 [ 2.2280873

In [13]:
for e in zip(model.layers[0].trainable_weights, model.layers[0].get_weights()):
    print('Param %s:\n%s' % (e[0],e[1]))
print('**************************************************************************')
for e in zip(model.layers[1].trainable_weights, model.layers[1].get_weights()):
    print('Param %s:\n%s' % (e[0],e[1]))

Param gru_1/kernel:
[[ 9.33152214e-02  7.07938969e-02 -3.47844735e-02  2.53624350e-01
   2.62463808e-01  1.41864687e-01  8.53539407e-02 -9.06321965e-03
  -1.02476627e-02 -1.54314354e-01  1.66473895e-01  5.23830578e-02
   1.91844851e-01  8.21528584e-02  4.49295975e-02 -1.40342489e-01
  -1.22833878e-01 -3.35400179e-02  2.46888828e-02  3.97469178e-02
   6.26918599e-02  6.77896440e-02 -6.93941116e-02  1.43367916e-01
   1.26597494e-01 -1.51089132e-01  1.85291254e-04 -1.45171136e-01
  -3.53385806e-02 -6.98544756e-02  9.36556309e-02 -9.43955034e-02
  -1.44611681e-02  3.77613716e-02 -1.62606701e-01  4.21235971e-02
  -1.19299009e-01  2.76639462e-01  2.09145218e-01  4.07007430e-03
   2.08005190e-01  6.90653324e-02 -1.24979168e-02  7.57546127e-02
  -6.57050982e-02  4.07019481e-02 -8.58648270e-02 -1.32628337e-01
  -1.34790391e-01 -4.00934136e-03  6.71319962e-02  8.44777450e-02
  -8.74350742e-02  1.29400730e-01 -8.85443017e-02 -1.49047926e-01
  -2.62996797e-02  1.29591674e-01  6.93774899e-04 -6.185

In [14]:
import keras.backend as K

In [15]:
for layer in model.layers:
    if 'GRU' in str(layer):
        print(layer)

In [16]:
import csv

model.summary()
#tobj = model.layers[0].output
#print(tobj[0].eval)

for layer in model.layers:
    if 'GRU' in str(layer):
        #w = layer.get_weights()
        #print(layer)
        print('Layer in GRU')
        #print('states[0] = {}'.format(K.get_value(layer.cell.states[0])))
        #print(layer.output.eval())
        print(layer.get_config())
        recurrent_kernel_h = (layer.cell.recurrent_kernel_h.eval())
        recurrent_kernel_z = (layer.cell.recurrent_kernel_z.eval())
        recurrent_kernel_r = (layer.cell.recurrent_kernel_r.eval())
        kernel_h = (layer.cell.kernel_h.eval())
        kernel_z = (layer.cell.kernel_z.eval())
        kernel_r = (layer.cell.kernel_r.eval())
        bias_h = (layer.cell.bias_h.eval())
        bias_z = (layer.cell.bias_z.eval())
        bias_r = (layer.cell.bias_r.eval())
        #print(layer.cell.ouput.eval)
        recurrent_kernel_h = layer.cell.recurrent_kernel_h.eval()
        with open('recurrent_kernel_h.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in recurrent_kernel_h:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        recurrent_kernel_r = layer.cell.recurrent_kernel_r.eval()
        with open('recurrent_kernel_r.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in recurrent_kernel_r:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        recurrent_kernel_z = layer.cell.recurrent_kernel_z.eval()
        with open('recurrent_kernel_z.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in recurrent_kernel_z:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        bias_h = layer.cell.bias_h.eval()
        with open('bias_h.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for value in bias_h:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        bias_r = layer.cell.bias_r.eval()
        with open('bias_r.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for value in bias_r:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        bias_z = layer.cell.bias_z.eval()
        with open('bias_z.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for value in bias_z:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        kernel_h = layer.cell.kernel_h.eval()
        with open('kernel_h.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in kernel_h:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        kernel_r = layer.cell.kernel_r.eval()
        with open('kernel_r.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in kernel_r:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n');
        kernel_z = layer.cell.kernel_z.eval() 
        with open('kernel_z.txt','wb') as csvfile:
            matrixwriter = csv.writer(csvfile, delimiter=' ')
            for row in kernel_z:
                for value in row:
                    np.savetxt(csvfile, np.array(value).reshape(1,), delimiter='\n'); 
                
       

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (1, 100)                  30600     
_________________________________________________________________
dense_1 (Dense)              (1, 1)                    101       
Total params: 30,701
Trainable params: 30,701
Non-trainable params: 0
_________________________________________________________________
Layer in GRU
{'trainable': True, 'stateful': True, 'return_state': False, 'units': 100, 'activation': 'tanh', 'bias_regularizer': None, 'kernel_regularizer': None, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'dtype': 'float32', 'use_bias': True, 'kernel_constraint': None, 'name': 'gru_1', 'batch_input_shape': (1, 1, 1), 'unroll': False, 'recurrent_dropout': 0.0, 'implementation': 1, 'recurrent_constraint': None, 'activity_regularizer': None, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 

In [17]:
import tensorflow as tf
def tanh(x):
    return np.tanh(x)

def sigmoid(x):
    return 0.5*tanh(0.5*x) + 0.5

def hard_sigmoid(x):
    #return K.hard_sigmoid(x)
    x = (x * 0.2) + 0.5
    z = 0.0 if x <= 0 else (1.0 if x >= 1 else x)
    return z


In [18]:
x = [0.98, 0.88]

#Param gru_1/kernel:
#[[ 0.21747403  0.79047525  0.4331279 ]
# [-0.6337419   0.8478408   0.81120795]]
#Param gru_1/recurrent_kernel:
#[[-0.5528964  -0.832763   -0.02848398]]
#Param gru_1/bias:
#[-0.7060377   0.          0.05893169]
#**************************************************************************
#Param dense_1/kernel:
#[[0.69523156]]
#Param dense_1/bias:
#[0.24978743]
#Param dense_1/kernel:
#[[1.075794  ]
# [0.88658786]]
#Param dense_1/bias:
#[0.01068966]
#Model predict
#[[0.6619987]]
#[[0.890478]]
#[[0.968756]]
#Param dense_1/kernel:
#[[ 0.36767545]
# [-0.53053135]
# [ 0.4265707 ]]
#Param dense_1/bias:
#[0.1615301]
#Model predict
#[[0.5351417]]
#[[0.68875945]]
#[[0.7260194]]
#Param dense_1/kernel:
#[[1.075794  ]
# [0.88658786]]
##Param dense_1/bias:
#[0.01068966]
#[[0.6619987]]
#[[0.890478]]
#[[0.968756]]
print(x)
print(bias_z,bias_r,bias_h)
print(kernel_z,kernel_r,kernel_h)
print(recurrent_kernel_z,recurrent_kernel_r,recurrent_kernel_h)


#[[0.53051203]]
#[[0.6143049]]
#Param dense_1/kernel:
#[[1.075794  ]
# [0.88658786]]
#Param dense_1/bias:
#[0.01068966]
#[[0.6619987]]
#[[0.890478]]
#[[0.968756]]

#Input 0.98

z = hard_sigmoid((x[0] * kernel_z[0][0]) + bias_z[0]) # w_1 is arranged vertically, 1 cell 1 state
r = hard_sigmoid((x[0] * kernel_r[0][0])  + bias_r[0])
h = tanh((x[0] * kernel_h[0][0]) + bias_h[0])
s = ( 1 - z) * h
s1 = s

z = hard_sigmoid((x[0] * kernel_z[0][1])  +   bias_z[1]) # w_1 is arranged vertically, 1 cell 1 state
r = hard_sigmoid((x[0] * kernel_r[0][1])  +  bias_r[1])
h = tanh((x[0] * kernel_h[0][1])  + bias_h[1])
s =  ( 1 - z) * h
s2 = s
print(s1 * W[0] + s2 * W[1] + B[0] )

#0.88 with states

z = hard_sigmoid((x[1] * kernel_z[0][0])  + s1 * recurrent_kernel_z[0][0] + s2 * recurrent_kernel_z[1][0] + bias_z[0]) # w_1 is arranged vertically, 1 cell 1 state
r = hard_sigmoid((x[1] * kernel_r[0][0])  + s1 * recurrent_kernel_r[0][0] + s2 * recurrent_kernel_r[1][0] + bias_r[0])

z1 = hard_sigmoid((x[1] * kernel_z[0][1])  + s1 * recurrent_kernel_z[0][1] + s2 * recurrent_kernel_z[1][1] +  bias_z[1]) # w_1 is arranged vertically, 1 cell 1 state
r1 = hard_sigmoid((x[1] * kernel_r[0][1])  + s1 * recurrent_kernel_r[0][1] + s2 * recurrent_kernel_r[1][1] + bias_r[1])

elem1 = r * s1
elem2 = r1 * s2

h = tanh((x[1] * kernel_h[0][0]) + (elem1 * recurrent_kernel_h[0][0] + elem2 * recurrent_kernel_h[1][0]) + bias_h[0])
s = z * s1 + ( 1 - z) * h
s11 = s


h = tanh((x[1] * kernel_h[0][1])  +(elem1 * recurrent_kernel_h[0][1] + elem2 * recurrent_kernel_h[1][1]) + bias_h[1])
s =  z1 * s2 + ( 1 - z1) * h

s21 = s
print(s11 * W[0] + s21 * W[1] + B[0] )


[0.98, 0.88]
[-0.16404371 -0.14445865 -0.14224316  0.23007293  0.20200521  0.15821634
 -0.02185588 -0.15268755 -0.14700201 -0.16216344  0.24840482  0.16462512
  0.1448659   0.1998829   0.17030296 -0.17033924 -0.17991309 -0.08686081
 -0.1526203  -0.13935752 -0.17699553 -0.16753492 -0.17718345  0.24885176
  0.19726078 -0.1617689  -0.18381736 -0.10111973 -0.15532908 -0.15361306
 -0.09557985 -0.14093316 -0.13278241  0.12782986 -0.14075732  0.1378138
 -0.20022553  0.26908773  0.23803376 -0.14938787 -0.00660238 -0.20094113
 -0.04852366 -0.08458431 -0.15042293 -0.01402915 -0.17035882 -0.1499555
 -0.20292218  0.17426516 -0.18051946 -0.18532209 -0.14540222  0.14692284
 -0.0248667  -0.13991565 -0.17440943 -0.05689294 -0.14017494 -0.19387175
 -0.18564852  0.22125171 -0.06561743  0.21642713  0.16611768 -0.19871548
  0.07539586  0.16707131  0.20255487  0.19825801 -0.01340905 -0.15162584
  0.08397608 -0.16602628  0.03901661  0.14666238  0.03449512 -0.17502734
  0.09683801  0.21834198 -0.14907023  0.

In [19]:
pr = (output_state * -1.82569075) + 0.15760025
print pr
print scaler.inverse_transform(pr)

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-19-b724e24faea1>, line 2)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("lstm_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("lstm_model.h5")
print("Saved model to disk")